Melissa Laurino <br>
Spring 2025

## In Class Activity - 2/20/25
### Create a practice Spotify database from a .csv file with no previous specification of tables (NO schema):

In [6]:
# Load necessary packages:
from sqlalchemy import create_engine, inspect, text # Database navigation
import sqlite3 # A second option for working with databases
import pandas as pd # Python data manilpulation

In [2]:
# Change working directory to explore mock Spotify database
import os
os.chdir('/Users/melissa/Documents/DSSA/DGW/Python_Spring2025/Spotify Extended Streaming History/')

In [42]:
# Load the .csv subset (Already cleaned with R in a previous semester)
csv_file = "spotify_subset_2012_2024.csv"  # Replace with your actual file path
df = pd.read_csv(csv_file)

# Create a SQLite database and engine
db_file = "spotify_data.db"
engine = create_engine(f"sqlite:///{db_file}")

# Store the dataframe in the database as a single table for easy practice
df.to_sql("spotify_history", con=engine, if_exists="replace", index=False)

/var/folders/_9/x6f1rdhs15d70lkp6829tnw80000gn/T/ipykernel_32221/124341830.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


202678

In [43]:
# Inspect the database to list the fields
inspector = inspect(engine)
columns = inspector.get_columns("spotify_history")

# Print column names
print("Columns in spotify_history table:")
for col in columns:
    print(col["name"], "-", col["type"])

Columns in spotify_history table:
ts - TEXT
platform - TEXT
ms_played - BIGINT
conn_country - TEXT
ip_addr - TEXT
master_metadata_track_name - TEXT
master_metadata_album_artist_name - TEXT
master_metadata_album_album_name - TEXT
spotify_track_uri - TEXT
episode_name - FLOAT
episode_show_name - FLOAT
spotify_episode_uri - FLOAT
audiobook_title - FLOAT
audiobook_uri - FLOAT
audiobook_chapter_uri - FLOAT
audiobook_chapter_title - FLOAT
reason_start - TEXT
reason_end - TEXT
shuffle - BOOLEAN
skipped - BOOLEAN
offline - BOOLEAN
offline_timestamp - FLOAT
incognito_mode - BOOLEAN
year - BIGINT


<br>
In class activity query guidelines: <br>
<br>
Query 1:<br>
Find the top 30 artists listened to in the year 2024.<br>
<br>
Query 2:<br>
Find the top 30 songs listened to in the year 2017.<br>
<br>
Query 3:<br>
Ask a question and create your own! <br>

### Query 1 Example: Top Artists

In [54]:
# SELECT the master_metadata_album_artist_name and the COUNT the number of times they appear as the play_count
# FROM the spotify_history database
# GROUP the results set BY the master_metadata_album_artist_name
# ORDER BY the play_count and sort the results in DESC order
# LIMIT the results for the top 30

query = text("""
                SELECT master_metadata_album_artist_name, COUNT (*) AS play_count
                FROM spotify_history
                GROUP BY master_metadata_album_artist_name
                ORDER BY play_count DESC
                LIMIT 30;
            """)

with engine.connect() as connection:
    top_artists = pd.read_sql(query, connection)

print(top_artists)

     master_metadata_album_artist_name  play_count
0                          Miley Cyrus        8895
1                        Ariana Grande        4536
2                          Marian Hill        4020
3                           Noah Cyrus        3673
4                               Hozier        3219
5                         Lana Del Rey        3169
6                            Lady Gaga        2988
7                       Nashville Cast        2923
8                       Meghan Trainor        2810
9                        Billie Eilish        2801
10                              Eminem        2691
11                        Taylor Swift        2587
12                 Panic! At The Disco        2507
13                      Britney Spears        2460
14                          Ed Sheeran        2437
15                      Mumford & Sons        2057
16                          Lord Huron        2051
17                      The Civil Wars        2033
18                       Amy Wi

### Query 1 Example - Top Artists in 2024 from 2012-2024

In [58]:
# SELECT the master_metadata_album_artist_name AS artist and SUM the ms_played and divide by 60000
# FROM the spotify_history table
# WHERE the year  is equal to 2024
# GROUP (Result set) BY the artist 
# ORDER (Result set) BY minute_played in DESC order
# LIMIT (Result set) by 30;

query = text("""
    SELECT master_metadata_album_artist_name AS artist, 
           SUM(ms_played) / 60000 AS minutes_played
    FROM spotify_history
    WHERE year = 2024
    GROUP BY artist
    ORDER BY minutes_played DESC
    LIMIT 30;
""")

with engine.connect() as connection:
    top_artists_2024 = pd.read_sql(query, connection)

print(top_artists_2024)

                 artist  minutes_played
0           Miley Cyrus            1307
1                Hozier            1159
2         Ariana Grande             840
3                Eminem             776
4             Lady Gaga             709
5     Sabrina Carpenter             706
6            Noah Kahan             623
7           Teddy Swims             483
8         Billie Eilish             477
9                   CIL             456
10                Dasha             434
11           Noah Cyrus             427
12     Steven Rodriguez             406
13              Beyoncé             366
14       Olivia Rodrigo             357
15       Mumford & Sons             343
16      Chris Stapleton             328
17        Chappell Roan             326
18         Taylor Swift             320
19         Lana Del Rey             317
20         Benson Boone             313
21  Panic! At The Disco             303
22                Kesha             291
23       Britney Spears             282


### Query 2 Example: Top Songs by Play Count - 2012-2024

In [103]:
# SELECT the master_metadata_track_name and COUNT(*) AS play_count
# FROM the spotify_history table
# GROUP (Result set) BY the master_metadata_track_name 
# ORDER (Result set) BY play_count in DESC order
# LIMIT (Result set) by 10;

query = text("""
    SELECT master_metadata_track_name, COUNT(*) AS play_count
    FROM spotify_history
    GROUP BY master_metadata_track_name
    ORDER BY play_count DESC
    LIMIT 10;
""")

with engine.connect() as connection:
    top_songs = pd.read_sql(query, connection)

print(top_songs)

        master_metadata_track_name  play_count
0                            River         540
1                           Malibu         521
2              All About That Bass         436
3                          7 rings         392
4                       Slide Away         390
5           When the Night is Over         387
6                    Fade Into You         386
7                             Down         381
8                  Angels Like You         366
9  It Ain’t Me (with Selena Gomez)         365


### Query 2 Example: Top 30 Songs by Minutes Played in 2017

In [78]:
# SELECT the master_metadata_track_name AS song and sum the ms_played and divide by 60000 to convert to minutes
# FROM the spotify_history table
# WHERE year = 2017
# GROUP (Result set) BY the song 
# ORDER (Result set) BY minutes_played in DESC order
# LIMIT (Result set) by 30;

query = text("""
    SELECT master_metadata_track_name AS song, 
           SUM(ms_played) / 60000 AS minutes_played
    FROM spotify_history
    WHERE year = 2017
    GROUP BY song
    ORDER BY minutes_played DESC
    LIMIT 30;
""")

with engine.connect() as connection:
    top_artists_2017 = pd.read_sql(query, connection)

print(top_artists_2017)

                                                 song  minutes_played
0                                              Malibu             677
1                                        Shape of You             489
2                     It Ain’t Me (with Selena Gomez)             414
3                                               Close             326
4                           Havana (feat. Young Thug)             277
5                                        Take Me Down             255
6                                              Anyway             227
7             Rockabye (feat. Sean Paul & Anne-Marie)             216
8                                    Week Without You             215
9                                        Love Someone             201
10                                      All Right Now             193
11                                            Rainbow             188
12                                            Praying             186
13                  

In [105]:
# SELECT the master_metadata_album_album_name AS album and COUNT all records as play_count
# FROM the spotify_history table
# WHERE year = 2024
# GROUP (Result set) BY the album 
# ORDER (Result set) BY play_count in DESC order
# LIMIT (Result set) by 5;

query = text("""
    SELECT master_metadata_album_album_name AS album, 
           COUNT(*) AS play_count
    FROM spotify_history
    WHERE year = 2024
    GROUP BY album
    ORDER BY play_count DESC
    LIMIT 5;
""")

with engine.connect() as connection:
    top_albums_2024 = pd.read_sql(query, connection)

print(top_albums_2024)

                                        album  play_count
0                            eternal sunshine         334
1                           What Happens Now?         194
2                     Endless Summer Vacation         168
3                                     Unheard         165
4  I've Tried Everything But Therapy (Part 1)         163


In [101]:
# SELECT the %H from the ts field AS hour and COUNT all records as play_count
# FROM the spotify_history table
# GROUP (Result set) BY the hour 
# ORDER (Result set) BY play_count in DESC order
# LIMIT (Result set) by 5;

query = text("""
            SELECT strftime('%H', ts) AS hour, COUNT(*) AS play_count
            FROM spotify_history
            GROUP BY hour
            ORDER BY play_count DESC
            LIMIT 5;
            """)

with engine.connect() as connection:
    top_hours = pd.read_sql(query, connection)

print(top_hours)

  hour  play_count
0   11       16978
1   12       16262
2   16       16183
3   15       14637
4   20       13014
